# Intro
In my this code,I try to use 4 pre-trainmode,5 Fold weight for each MRI-TYPE.And,I converte Test Data to TFRecord.  
Because,without TFRecord,Notebook is timeout after submittion.  
**Version 2.0**
Display Images each MRI-TYPES

In [ ]:
!pip install ../input/tensorflowprobability0122/tensorflow_probability-0.12.2-py2.py3-none-any.whl
!pip3 install ../input/kerasapplications -q
!pip3 install ../input/image-classifiers/image_classifiers-1.0.0-py3-none-any.whl

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import json
import argparse
import datetime
import random
from pathlib import Path
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from sklearn.metrics import log_loss
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import math
import gc
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
from classification_models.keras import Classifiers

# Load Data
weight-20210903-full" is weigth. 4 MRI-TYPE * 5 Fold * 4 model *2 =  160 files

In [ ]:
df_preds = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
weightdatapath = Path("../input/weight-20210903-full")
testdatapaht = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test"

In [ ]:
height = 256
width = 256
channel = 3
batch_size = 16
seed = 46
views = ['FLAIR', 'T1w', 'T1wCE', 'T2w']

MRI_TYPE = 0

In [ ]:
def set_seed(seed=200):
    tf.random.set_seed(seed)

    np.random.seed(seed)

    random.seed(seed)

    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(seed)

In [ ]:
def load_imgs(idx,view, ignore_zeros=True):
    imgs = {}
    
    save_ds = []
    dir_path = os.walk(os.path.join(
        testdatapaht, idx, view
    ))
    for path, subdirs, files in dir_path:
        for name in files:
            image_path = os.path.join(path, name) 
            pyds = pydicom.filereader.dcmread(image_path)
            slope = float(pyds.RescaleSlope)
            intercept = float(pyds.RescaleIntercept)
            img = intercept + pyds.pixel_array * slope
            img = cv2.resize(img,[height,width])
            save_ds.append(np.array(img))
    if len(save_ds) == 0:
        save_ds = np.zeros((1,256,256))
    imgs = np.array(save_ds)
    return imgs

# Convert Gray-Scale images to 3 Channel images
DICOM is Gray-Scale.But Pre-Train models' input is 3channel.  
So,I divided the whole into one-third and get mean each part.  
And I multiplied 0.3 or 0.4 so that total is 1.  
I call this my method **"Hazigin Method"** :-)

In [ ]:
dim = (height,width)
t_imgs = np.empty((channel, *dim))
def data_generation(ID,view,is_Train=True):
    idx = str(ID).zfill(5)
    imgs = load_imgs(idx,view, ignore_zeros=False)
    t_size = imgs.shape
    t_a = math.ceil(t_size[0] / 3)
    t_img =imgs[:t_a]
    t_imgs[0] = t_img.mean(axis=0) * 0.3
    t_img =imgs[t_a:t_size[0] - t_a]
    t_imgs[1] = t_img.mean(axis=0) * 0.4
    t_img =imgs[t_size[0] - t_a:]
    t_imgs[2] = t_img.mean(axis=0) * 0.3
    img_ = t_imgs.transpose(1,2,0)
    return img_

# Argumentation
This argumentation is to that Images are too dark are bright, and  too bright are dark.

In [ ]:
def argument_image_val(img):
    hsv_tf = tf.image.rgb_to_hsv(img)
    a = tf.transpose(hsv_tf,[2,0,1])
    a = tf.reshape(a[2],[256,256]) 
    t_v = tf.cond( tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)>200,lambda:tf.cast(0,tf.float32),lambda:tf.cast((200 - tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95))*0.04,tf.float32))
    t_c = tf.cond( tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)>200,lambda:tf.cast(0,tf.float32),lambda:tf.cast((200 - tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95))*0.005,tf.float32))   
    t_v_o = tf.cond( tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)>300,lambda:tf.cast((tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95) - 300)*0.5,tf.float32),lambda:tf.cast(0,tf.float32))

    img = tf.image.adjust_brightness(img, delta=(2.8+ t_v - t_v_o))
    aa = tf.cond( (0.3 + t_c) < 2,lambda:tf.cast((0.3 + t_c),tf.float32),lambda:tf.cast(2,tf.float32))   

    img = tf.image.adjust_contrast(img, tf.cast(aa,tf.float32))
    img = tf.image.adjust_saturation(img, 1.2)
    img = tf.cast(img, tf.float32) / 255.0
    return img

In [ ]:
def _bytes_feature(value):
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() 
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
def serialize_example_test(feature0):
  feature = {
      'image': _bytes_feature(feature0.tobytes())
  }
  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

# TFRecord
Convert Test Data to TFRecord for each MRI type

In [ ]:
for i in range(4):
    with tf.io.TFRecordWriter(str("./") + str("brain_test_" + views[i] + ".tfrec")) as writer:
        for x in df_preds["BraTS21ID"]:
            img = data_generation(x,views[i],False)
            example = serialize_example_test(
                img)
            writer.write(example)

In [ ]:
def deserialize_example(serialized_string):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string)
    }
    parsed_record = tf.io.parse_single_example(serialized_string, image_feature_description)
    image = tf.io.decode_raw(parsed_record['image'], tf.float64)
    image = tf.reshape(image,[height,width,channel])
    return image

In [ ]:
def _parse_image_function(example_proto): 
  return tf.io.parse_single_example(example_proto, image_feature_description)

# Disp Images[FLAIR]
Disp FLAIR Image with brightness.

In [ ]:
MRI_TYPE = 0
raw_image_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec"))

image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string)
}

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

fig = plt.figure(figsize=(35, 20))
page = 1
i=0
for idx in range(page*15,(page + 1)*15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.tick_params(labelbottom=False, bottom=False)
    ax.tick_params(labelleft=False, left=False)
    for parsed_record in parsed_image_dataset.take(idx):
        img = tf.io.decode_raw(parsed_record['image'], tf.float64)
        img = tf.reshape(img,[256,256,3])

    hsv_tf = tf.image.rgb_to_hsv(img)
    a = tf.transpose(hsv_tf,[2,0,1])
    a = tf.reshape(a[2],[256,256]) 
    b  = tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)

    img = argument_image_val(img)
    ax.imshow(img)
    ax.set_title(str(idx) + ":" + str(b.numpy()),color="Black")
    i = i+1
fig.show()

# Disp Images[T1w]
Disp T1w Image with brightness.

In [ ]:
MRI_TYPE = 1
raw_image_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec"))

image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string)
}

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

fig = plt.figure(figsize=(35, 20))
page = 2
i=0
for idx in range(page*15,(page + 1)*15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.tick_params(labelbottom=False, bottom=False)
    ax.tick_params(labelleft=False, left=False)
    for parsed_record in parsed_image_dataset.take(idx):
        img = tf.io.decode_raw(parsed_record['image'], tf.float64)
        img = tf.reshape(img,[256,256,3])

    hsv_tf = tf.image.rgb_to_hsv(img)
    a = tf.transpose(hsv_tf,[2,0,1])
    a = tf.reshape(a[2],[256,256]) 
    b  = tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)

    img = argument_image_val(img)
    ax.imshow(img)
    ax.set_title(str(idx) + ":" + str(b.numpy()),color="Black")
    i = i+1
fig.show()

# Disp Images[T1wCE]
Disp T1wCE Image with brightness.

In [ ]:
MRI_TYPE = 2
raw_image_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec"))

image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string)
}

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

fig = plt.figure(figsize=(35, 20))
page = 3
i=0
for idx in range(page*15,(page + 1)*15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.tick_params(labelbottom=False, bottom=False)
    ax.tick_params(labelleft=False, left=False)
    for parsed_record in parsed_image_dataset.take(idx):
        img = tf.io.decode_raw(parsed_record['image'], tf.float64)
        img = tf.reshape(img,[256,256,3])

    hsv_tf = tf.image.rgb_to_hsv(img)
    a = tf.transpose(hsv_tf,[2,0,1])
    a = tf.reshape(a[2],[256,256]) 
    b  = tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)

    img = argument_image_val(img)
    ax.imshow(img)
    ax.set_title(str(idx) + ":" + str(b.numpy()),color="Black")
    i = i+1
fig.show()

# Disp Images[T2w]
Disp T2w Image with brightness.

In [ ]:
MRI_TYPE = 3
raw_image_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec"))

image_feature_description = {
    'image': tf.io.FixedLenFeature([], tf.string)
}

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)

fig = plt.figure(figsize=(35, 20))
page = 4
i=0
for idx in range(page*15,(page + 1)*15):
    ax = fig.add_subplot(3, 5, i+1)
    ax.tick_params(labelbottom=False, bottom=False)
    ax.tick_params(labelleft=False, left=False)
    for parsed_record in parsed_image_dataset.take(idx):
        img = tf.io.decode_raw(parsed_record['image'], tf.float64)
        img = tf.reshape(img,[256,256,3])

    hsv_tf = tf.image.rgb_to_hsv(img)
    a = tf.transpose(hsv_tf,[2,0,1])
    a = tf.reshape(a[2],[256,256]) 
    b  = tfp.stats.percentile(tf.experimental.numpy.ravel(a),q=95)

    img = argument_image_val(img)
    ax.imshow(img)
    ax.set_title(str(idx) + ":" + str(b.numpy()),color="Black")
    i = i+1
fig.show()

In [ ]:
class ClassificationModel_senet154(tf.keras.Model):
    def __init__(self):
        super(ClassificationModel_senet154, self).__init__()
        ResNet18, preprocess_input = Classifiers.get('senet154')
        base_model = ResNet18((height,width, channel), include_top=False, weights='../input/image-classifiers/senet154_imagenet_1000_no_top.h5')
        base_model.trainable = False
        self.efficient_net = base_model
        self.avgpooling = tf.keras.layers.GlobalAveragePooling2D()
        self.dence64 = tf.keras.layers.Dense(64, kernel_initializer='lecun_normal',activation='relu')
        self.dence32 = tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='relu')
        self.efficient_net = base_model
        self.dropoup = tf.keras.layers.Dropout(0.2)
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

                
    def call(self, input_tensor, training=True):
        x = self.efficient_net(input_tensor)
        #x = self.dence64(x)
        x = self.avgpooling(x)
        x = self.dence64(x)
        x = self.dropoup(x)
        x = self.dence32(x)
        return self.dence1(x) 

In [ ]:
class ClassificationModel_ResNet152V2(tf.keras.Model):
    def __init__(self):
        super(ClassificationModel_ResNet152V2, self).__init__()
        base_model = tf.keras.applications.resnet_v2.ResNet152V2(weights="../input/image-classifiers/resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5",include_top=False,pooling='avg',input_shape=(height,width, 3))
        base_model.trainable = False
        self.efficient_net = base_model
        self.dence64 = tf.keras.layers.Dense(64, kernel_initializer='lecun_normal',activation='relu')
        self.dence32 = tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='relu')
        self.efficient_net = base_model
        self.dropoup = tf.keras.layers.Dropout(0.2)
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

                
    def call(self, input_tensor, training=True):
        x = self.efficient_net(input_tensor)
        x = self.dence64(x)
        x = self.dropoup(x)
        x = self.dence32(x)
        return self.dence1(x) 

In [ ]:
class ClassificationModel_seresnext101(tf.keras.Model):
    def __init__(self):
        super(ClassificationModel_seresnext101, self).__init__()
        ResNet18, preprocess_input = Classifiers.get('seresnext101')
        base_model = ResNet18((height,width, channel), include_top=False, weights='../input/image-classifiers/seresnext101_imagenet_1000_no_top.h5')
        base_model.trainable = False
        self.efficient_net = base_model
        self.avgpooling = tf.keras.layers.GlobalAveragePooling2D()
        self.dence64 = tf.keras.layers.Dense(64, kernel_initializer='lecun_normal',activation='relu')
        self.dence32 = tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='relu')
        self.efficient_net = base_model
        self.dropoup = tf.keras.layers.Dropout(0.2)
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

                
    def call(self, input_tensor, training=True):
        x = self.efficient_net(input_tensor)
        x = self.avgpooling(x)
        x = self.dence64(x)
        x = self.dropoup(x)
        x = self.dence32(x)
        return self.dence1(x) 

In [ ]:
class ClassificationModel_seresnet152(tf.keras.Model):
    def __init__(self):
        super(ClassificationModel_seresnet152, self).__init__()
        ResNet18, preprocess_input = Classifiers.get('seresnet152')
        base_model = ResNet18((height,width, channel), include_top=False, weights='../input/image-classifiers/seresnet152_imagenet_1000_no_top.h5')
        base_model.trainable = False
        self.efficient_net = base_model
        self.avgpooling = tf.keras.layers.GlobalAveragePooling2D()
        self.dence64 = tf.keras.layers.Dense(64, kernel_initializer='lecun_normal',activation='relu')
        self.dence32 = tf.keras.layers.Dense(32, kernel_initializer='lecun_normal',activation='relu')
        self.efficient_net = base_model
        self.dropoup = tf.keras.layers.Dropout(0.2)
        self.dence1 = tf.keras.layers.Dense(1, activation='sigmoid')

                
    def call(self, input_tensor, training=True):
        x = self.efficient_net(input_tensor)
        x = self.avgpooling(x)
        x = self.dence64(x)
        x = self.dropoup(x)
        x = self.dence32(x)
        return self.dence1(x)

In [ ]:
loss_func = tf.keras.losses.BinaryCrossentropy(from_logits=True)
opt = tf.keras.optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999)

In [ ]:
weight_Model=["ResNet152V2","senet154","seresnet152","seresnext101"]
t_pre=[] #For each MRI type data
m_pre=[] #For each pre-train mode data
f_pre=[] #For each Fold data

# Choice Models
The model to be applied to each MRI is selected from the top two models with good CV scores.

In [ ]:
#FLAIR
MRI_TYPE = 0
for m in [1,3]: #Select MODELs
    if m == 0:
        model = ClassificationModel_ResNet152V2()
    elif m == 1:
        model = ClassificationModel_senet154()
    elif m == 2:
        model = ClassificationModel_seresnet152()
    elif m == 3:
        model = ClassificationModel_seresnext101()

    model.compile(
        optimizer=opt, 
        loss=loss_func,
        metrics=['accuracy']
        )
    
    for f in range(5): #Fold 
        test_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec")).map(deserialize_example).map(argument_image_val).batch(batch_size)
        t_weight = "weight-" + weight_Model[m] + "-" + views[MRI_TYPE] + "_fold_0" +str(f) + "-0903.ckpt"
        model.load_weights(Path(weightdatapath,t_weight))
        f_pre.append(model.predict(test_dataset, batch_size=batch_size))
        tf.keras.backend.clear_session()

    pre = np.array(f_pre)
    m_pre.append(pre.mean(axis=0))
    f_pre.clear()

pre = np.array(m_pre)
t_pre.append(pre.mean(axis=0))
m_pre.clear()
del model
gc.collect()

In [ ]:
#T1w
MRI_TYPE = 1
f_pre.clear()
for m in [0,1]: #Select MODELs
    if m == 0:
        model = ClassificationModel_ResNet152V2()
    elif m == 1:
        model = ClassificationModel_senet154()
    elif m == 2:
        model = ClassificationModel_seresnet152()
    elif m == 3:
        model = ClassificationModel_seresnext101()

    model.compile(
        optimizer=opt, 
        loss=loss_func,
        metrics=['accuracy']
        )
    
    for f in range(5): #Fold 
        test_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec")).map(deserialize_example).map(argument_image_val).batch(batch_size)
        t_weight = "weight-" + weight_Model[m] + "-" + views[MRI_TYPE] + "_fold_0" +str(f) + "-0903.ckpt"
        model.load_weights(Path(weightdatapath,t_weight))
        f_pre.append(model.predict(test_dataset, batch_size=batch_size))
        tf.keras.backend.clear_session()

    pre = np.array(f_pre)
    m_pre.append(pre.mean(axis=0))
    f_pre.clear()

pre = np.array(m_pre)
t_pre.append(pre.mean(axis=0))
m_pre.clear()
del model
gc.collect()

In [ ]:
#T1wCE
MRI_TYPE = 2
f_pre.clear()
for m in [1,2]: #Select MODELs
    if m == 0:
        model = ClassificationModel_ResNet152V2()
    elif m == 1:
        model = ClassificationModel_senet154()
    elif m == 2:
        model = ClassificationModel_seresnet152()
    elif m == 3:
        model = ClassificationModel_seresnext101()

    model.compile(
        optimizer=opt, 
        loss=loss_func,
        metrics=['accuracy']
        )
    
    for f in range(5): #Fold 
        test_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec")).map(deserialize_example).map(argument_image_val).batch(batch_size)
        t_weight = "weight-" + weight_Model[m] + "-" + views[MRI_TYPE] + "_fold_0" +str(f) + "-0903.ckpt"
        model.load_weights(Path(weightdatapath,t_weight))
        f_pre.append(model.predict(test_dataset, batch_size=batch_size))
        tf.keras.backend.clear_session()

    pre = np.array(f_pre)
    m_pre.append(pre.mean(axis=0))
    f_pre.clear()

pre = np.array(m_pre)
t_pre.append(pre.mean(axis=0))
m_pre.clear()
del model
gc.collect()

In [ ]:
#T2w
MRI_TYPE = 3
f_pre.clear()
for m in [0,2]: #Select MODELs
    if m == 0:
        model = ClassificationModel_ResNet152V2()
    elif m == 1:
        model = ClassificationModel_senet154()
    elif m == 2:
        model = ClassificationModel_seresnet152()
    elif m == 3:
        model = ClassificationModel_seresnext101()

    model.compile(
        optimizer=opt, 
        loss=loss_func,
        metrics=['accuracy']
        )
    
    for f in range(5): #Fold 
        test_dataset = tf.data.TFRecordDataset(str("./") + str("brain_test_" + views[MRI_TYPE] + ".tfrec")).map(deserialize_example).map(argument_image_val).batch(batch_size)
        t_weight = "weight-" + weight_Model[m] + "-" + views[MRI_TYPE] + "_fold_0" +str(f) + "-0903.ckpt"
        model.load_weights(Path(weightdatapath,t_weight))
        f_pre.append(model.predict(test_dataset, batch_size=batch_size))
        tf.keras.backend.clear_session()

    pre = np.array(f_pre)
    m_pre.append(pre.mean(axis=0))
    f_pre.clear()

pre = np.array(m_pre)
t_pre.append(pre.mean(axis=0))
m_pre.clear()
del model
gc.collect()

In [ ]:
pre = np.array(t_pre)
finpre = pre.mean(axis=0)

subfilename = "submission.csv"
sub = pd.DataFrame(finpre, columns=['MGMT_value'])

df_preds["MGMT_value"] = sub

df_preds.to_csv(
    subfilename,
    index=False
    )

# Request
**If you find this Code useful, please upvote !!**